In [1]:
import spacy
nlp = spacy.load('en_core_web_sm')


def find_persons(text):
    # Create Doc object
    doc2 = nlp(text)

    # Identify the persons
    persons = [ent.text for ent in doc2.ents if ent.label_ == 'PERSON']

    # Return persons
    return persons

def hasStopWord(x):
    stopWords = set(['Also', 'Sunday', 'Happy', 'Show', 'Best', 'Current', 'Tv', 'Award', 'No', 'It', 'What', 'Are', 'To', 'Win', 'My', \
        'Will', 'On', 'This', 'So', 'Here', 'Go', 'Actors', 'And', 'Who', 'Have', 'These', 'Is', 'Did', 'Be', 'She', 'He', 'With', 'The'])
    for word in stopWords:
        if word in x:
            return True
    return False

def isHypothetical(text):
    if '?' in text or 'last year' in text or 'hope' in text or 'hoping' in text or 'bet' in text or \
        'think' in text or 'will' in text or 'predict' in text or 'going to' in text or 'gonna' in text:
        return True
    return False

def isHistorical(text):
    if re.findall(r"\d\d\d\d", text) and '2015' not in text and '2014' not in text:
        return True
    return False

def isReasonable(text):
    if not isHypothetical(text.lower()) and not isHistorical(text.lower()):
        return True
    return False

def isWinningTweet(text, awardIndicators, trips = []):
    """
        <text> = tweet text
        <award> = list of indicators that belong to a given award
    """
    text = text.lower()
    winIndicators = ['won', 'win', 'congrat', 'goes to', 'went to', 'snag', 'takes home', 'took home']
    for i in winIndicators: #we want an indication of win
        if i in text:
            for a in awardIndicators:
                if a not in text:
                    return False # we want an indication of our relevant award
            for t in trips:
                if t in text:
                    return False # we do not want indication of a tripword
            return True
    # does not contain an indicator of a winning tweet
    return False

In [2]:
import json
import re
import csv

# Opening JSON file
f = open('gg2015.json')
 
# returns JSON object as
# a dictionary
data = json.load(f)

print("loaded file")

tt = []
# print first 100 tweets
for tweet in data:
    # print(tweet['text'], data[99]['user']['screen_name'])
    tt.append(tweet['text'])
del data

tNamesSet = set(())

for t in tt:
    # print(t)
    candNames = re.findall(r'\b[A-Z][a-z]+ [A-Z][a-z]+', t)
    for n in candNames:
        if not hasStopWord(n):
            tNamesSet.add(n)

tNames = list(tNamesSet)
print("number of regex names found:", len(tNames))
# print(tNames)
f.close()

loaded file
number of regex names found: 34092


In [4]:
class award:
    def __init__(self, keywords = [], tripwords = []):
        self.keywords = keywords
        self.tripwords = tripwords
        self.winner = ""

In [5]:
bestActor = award(["actor", "comedy", "motion", "best", "musical"],["supporting", "nom", "tv", "television", "drama"])
print("keys", bestActor.keywords)
print("trips:", bestActor.tripwords)
print("winner:", bestActor.winner)

keys ['actor', 'comedy', 'motion', 'best', 'musical']
trips: ['supporting', 'nom', 'tv', 'television', 'drama']
winner: 


In [3]:
# create a sample award indicators list "best performance by an actor in a motion picture - comedy or musical": {
bestActor = ["actor", "comedy", "motion", "best", "musical"]
trips = ["supporting", "nom", "tv", "television", "drama"]
#if this needs to be automated: find the closest award names to a given award: i.e. (best actress in a motion picture drama)
# would be similar to (best actress in a motion picture comedy or musical) and (best actress in a tv movie) or whatever,
# then pick out the words that are in similar award names but not in the one of interest. those are our tripwords.

# iterate over tweets and add counts to those actors that won an award
winningTweets = []
for t in tt:
    if isReasonable(t) and isWinningTweet(t, bestActor, trips):
        winningTweets.append(t)

print("number of winning tweets found for : ", bestActor, " = ", len(winningTweets))

candWinners = {}
actorNamesSet = set(())

for t in winningTweets[:1000]:
    people = find_persons(t)
    # print(t)
    # print(people)
    for p in people:
        if '@' in p or 'RT' in p or 'golden' in p:
            people = people.remove(p)
    if not people:
        continue
    # print(people)
    ats = re.findall(r'@[a-zA-Z_\-\.]+', t)
    # if ats:
        # print(ats)
    for p in people:
        if p in actorNamesSet:
            candWinners[p] += 1
        else:
            candWinners[p] = 1
        actorNamesSet.add(p)
winnerCounts = (sorted(candWinners.items(), key=lambda item: 1/item[1]))
print(winnerCounts[0][0])

number of winning tweets found for :  ['actor', 'comedy', 'motion', 'best', 'musical']  =  1487
Michael Keaton
